<a href="https://colab.research.google.com/github/hericax/Pos/blob/master/projeto_3/aed_projeto_3(colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto #3 - Meu primeiro projeto de IA

Antes de começar, leia as [Instruções](https://github.com/thvmm/pos-ds-ia/blob/master/projeto_3/README.md) e os [Critérios de Avaliação](https://github.com/thvmm/pos-ds-ia/blob/master/projeto_3/README.md).


### 1) **(5%)** Qual a base escolhida e qual seu interesse nela (podem ser mais de uma)? Descrição básica do conjunto de dados escolhido pelo aluno (1 parágrafo).

*Indique o link da base no Kaggle e explique em um parágrafo curto por que essa base é interessante para você. Quais são as características básicas da base?*

Ex: Estou trabalhando com o histório de vendas de um varejo e uma outra base do mesmo varejista das suas campanhas de marketing realizadas em veiculos de comunicação.

A base escolhida é de filmes - https://www.kaggle.com/rounakbanik/the-movies-dataset

O conjunto de dados consiste em filmes lançados em ou antes de julho de 2017. Os pontos de dados incluem elenco, equipe, palavras-chave da trama, orçamento, receita, pôsteres, datas de lançamento, idiomas, empresas de produção, países, contagem de votos no TMDB e média de votos. 
O conjunto possui 6 arquivo, o que foi utilizado no trabalho é o movies_metadata.csv, que contém informações sobre 45.000 filmes em destaque no conjunto de dados Full MovieLens. Este conjunto de dados também possui arquivos contendo 26 milhões de classificações, de 270.000 usuários para todos os 45.000 filmes. 

As classificações estão em uma escala de 1 a 5 e foram obtidas no site oficial do GroupLens.

### 2) **(5%)** Que tipo de problema podemos solucionar com ela? Liste ao menos 3 hipóteses sobre seus dados, caso sua base seja muito restrita converse com os professores antes de seguir.
- Quais são suas hipoteses sobre os dados? O que vc espera descobrir com esse estudo?

Ex: Hipoteses válidas seriam:
- As pessoas fazem mais compras nos dias utéis ou finais de semana?
- Existe diferença entre o padrão de compra de segunda ou terça feira?
- O horário de pico influencia no padrão de compra dos clientes?
- Quais produtos são comprados mais juntos? Existe diferença se observarmos a idade do cliente?
- Qual o efeito de uma campanha de marketing nas vendas?


Hipóteses: 

1 - Quais são os gêneros de filmes melhor avaliados? 

2 - Existe relação dos filmes mais bem avaliados para a empresa que o produziu? 

3 - Há influência dos idiomas disponíveis no filmes na avaliação dos filmes?


### 3) **(20%)** Preparação de dados

Hora de deixar a base perfeita para se trabalhar. Limpe os dados, integre com outras fontes e transforme-o para ficar pronto para se trabalhar! No fim, mostre um antes e depois de seu dataframe.

In [ ]:
# IMPORTANDO BIBLIOTECAS

# básicas
import pandas as pd
import numpy as np

# visualização
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import missingno as msno #! pip install missingno

# pré-processamento
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# modelos


In [ ]:
# Leitura do dataset
filmes = pd.read_csv('C:/Users/Herica/Desktop/Herica/cursoIACD/bases/filmes/movies_metadata.csv')

In [ ]:
# Pré-visaulização do dataset
filmes.head(3)

In [ ]:
# tamanho da base
print('Número de linhas e número de colunas:', filmes.shape)

In [ ]:
# fazendo uma cópia do dataset para ser trabalhado
df = filmes.copy()

In [ ]:
# tamanho da base
print('Número de linhas e número de colunas:', df.shape)

In [ ]:
df.info()

# RENOMEANDO AS FEATURES

In [ ]:
df = df.rename(columns={'id':'id_filme','adult': "classificacao", 'budget': 'despesas', 'genres': 'genero', 'original_language': 'lingua_original', 'original_title': 'titulo_original', 'overview': 'descricao', 'popularity': 'popularidade', 'production_companies': 'cia_producao', 'production_countries': 'pais_producao', 'release_date': 'data_lancamento', 'revenue': 'receita', 'runtime': 'duracao', 'spoken_languages': 'idiomas_falados', 'title': 'titulo', 'vote_average': 'media_votos', 'vote_count': 'qtde_votos','video': 'video'})

In [ ]:
# validando o nome das features
df.columns

# EXCLUSÃO DE FEATURES COM DADOS QUE NÃO SERÃO TRABALHADOS

In [ ]:
#dropar colunas consideradas irrelevantes na primeira análise ou que possuem muitos valores nulos
df.drop(['belongs_to_collection'],axis = 1, inplace = True)
df.drop(['homepage'],axis = 1, inplace = True)
df.drop(['poster_path'],axis = 1, inplace = True)     
df.drop(['tagline'],axis = 1, inplace = True)

df.drop(['imdb_id'], inplace=True, axis=1)

In [ ]:
# tamanho da base
print('Número de linhas e número de colunas:', df.shape)

# DESCRIÇÃO DAS FEATURES

'adulto' (booleano) - se o filme é exclusivo para adulto ou não . Contém dois valores: 'false' = 1, 'true' = 0

'despesas' (numérico) - possui o valor gasto na produção e divulgação dos vídeos 

'genero' (dicionário / texto) - contém qual ou quais são os gênceros de cada filme. Um filme pode ter um ou gêneros. Contém 29 valores possíveis. 

'id' (numérico) - número que identifica o filme 

'imdb_id'  (alfanumérico)- número unico que identifica o filme na base do imdb 

'lingua_original' (texto) - qual é o idioma original do filme produzido. Possui 90 valores possíveis.

'titulo_original' (texto) - titulo do filme 

'descricao' (texto) - descrição do filme 

'popularidade' (numérico)- qual é a popularidade do filme 

'cia_producao' (dicionário / texto) - qual(is) foi a companhia(s) que produziu o filme. Possui 19368 valores possíveis. Cada filme pode ter no mínimo um.

'pais_producao' (dicionário / texto) - quais foram os países onde os filmes foram gravados. Possui 155 valores possíveis. Cada filme pode ter no mínimo um.

'data_lancamento' (texto) - qual foi a data de lançamento do filme 

'receita'  (numérico) -  qual foi a receita obtida com o filme 

'duracao'  (numérico) - qual é a duração do filme em minutos 

'idiomas_falados' (dicionário/ texto) - quais são os idiomas que o filme possui tradução. Possui 73 valores possíveis.

'status' (texto) - status do filmes. Possui 7 valores possíveis: 'Released', nan, 'Rumored', 'Post Production', 'In Production', 'Planned', 'Canceled'

'slogan' (texto) - frase que define o filme. 

'titulo' (texto) - qual é o título do filme 

'video' (booleano) - se tem o video ou não do traile. Possui 3 valores possíveis: False, True, nan

'media_votos'  (numérico)- qual é a média de votos dada pelos usuários 

'qtde_votos'  (numérico)- qual foi a quantidade de votos

# VERIFICANDO DADOS NULOS GLOBAIS
Cada feature será analisada

In [ ]:
df.isnull().sum()

In [ ]:
# configuração do plot
figsize = (10, 7)
matplotlib.rcParams['figure.figsize'] = figsize


# visualização de dados faltantes
msno.bar(df, figsize=figsize,)

In [ ]:
# visualização de dados faltantes
msno.matrix(df, figsize=figsize)

In [ ]:
df.isnull().sum()

In [ ]:
# lista dos filmes sem media_votos
df[df['media_votos'].isnull()]

In [ ]:
# deletando os registros sem media de votos
i = df.shape[0]
df.dropna(subset=['media_votos'], inplace = True)
f = df.shape[0]
print ('Qtde de registros excluídos: ', i-f)

In [ ]:
df.isnull().sum()

In [ ]:
# lista dos filmes sem descricao
df[df['descricao'].isnull()]

In [ ]:
# atualizando os dados faltantes
df.descricao = df.descricao.fillna('Sem descrição')

In [ ]:
df.isnull().sum()

In [ ]:
# lista dos filmes sem lingua original
df[df['lingua_original'].isnull()]

In [ ]:
# Atualizando o valor da lingua original, baseando no pais de produção do filmes

#Áutria, Austrália e EUA: indice: 44576     41872   19574   21602   22832   32141 -> 'en'
df.at[44576, 'lingua_original'] = 'en'
df.at[41872, 'lingua_original'] = 'en'
df.at[19574, 'lingua_original'] = 'en'
df.at[21602, 'lingua_original'] = 'en'
df.at[22832, 'lingua_original'] = 'en'
df.at[32141, 'lingua_original'] = 'en'

#'Czech Republic: indice: 37407 -> 'CZ'
df.at[37407, 'lingua_original'] = 'CZ'

#France: indice: 44655  e 44057 -> 'fr'         
df.at[44655, 'lingua_original'] = 'fr'
df.at[44057, 'lingua_original'] = 'fr'

In [ ]:
# validando alterações na feature lingua_original
df[df['lingua_original'].isnull()]

In [ ]:
# deletando o restante
df.dropna(subset=['lingua_original'], inplace = True)

In [ ]:
# deletando os registros sem data_lancamento e status
df.dropna(subset=['data_lancamento','status'], inplace=True)

In [ ]:
df.isnull().sum()

# VERIFICANDO REGISTROS DUPLICADOS
Fearure  ID que será utilizada como chave

In [ ]:
# encontrando os registros duplicados no ID
df_duplicado = df.groupby(['id_filme']).size().reset_index(name='count')
df_duplicado.query('count > 1')

In [ ]:
# deletando os filmes com mesmo ID, amntendo o primeiro
df.drop_duplicates(subset='id_filme', keep='first', inplace = True)

In [ ]:
# encontrando filmes diferentes com mesmo título
df_duplicado = df.groupby(['titulo']).size().reset_index(name='count')
df_duplicado.query('count > 1')

# TIPOS DE FEATURES

A) Variáveis categóricas: adulto, genero , lingua_original, cia_producao, pais_producao, idiomas_falado, status, slogan, titulo, titulo_original, descricao, video

B) Variáveis discretas: despesas, id, data_lancamento, popularidade, receita, duracao(minutos), media_votos, qtde_votos  

In [ ]:
df.dtypes

# PRÉ-PROCESSANDO OS DADOS

In [ ]:
## FEATURE MÉDIA DE VOTO - TARGET

In [ ]:
df.media_votos.head(5)

In [ ]:
df.media_votos.describe()

In [ ]:
df.media_votos.unique()

## FEATURE CLASSIFICAÇÃO

In [ ]:
df.classificacao.unique()

In [ ]:
df.classificacao.value_counts()

In [ ]:
#Existem somente 8 filmes com classificação True, portanto esta feature se tornou irrelevante e será excluída.
df.drop(['classificacao'], inplace=True, axis=1)

# Validando a exclusão
df.columns

## FEATURE DESPESAS

In [ ]:
df.despesas.describe()

In [ ]:
print ('Qtde de registros únicos: ', len(df.despesas.unique()))

In [ ]:
df.despesas.value_counts()

In [ ]:
#Conforme observamos, dos 45463 registros da base, 36391 não possui valores de despesas, portanto a feature foi considerada irrelevante e excluída.

df.drop(['despesas'], inplace=True, axis=1)

# Validando a exclusão
df.columns

## FEATURE GENERO

In [ ]:
df.genero.describe()

In [ ]:
df[["genero", "titulo_original"]].head(3)

In [ ]:
# Função para transformar a feature em uma lista de generos ordenados
from ast import literal_eval
df['genero'] = df['genero'].apply(literal_eval) # verifica se todos os dados são do mesmo tipo
#Função para pegar os nomes dos gêneros, caso exista.
def get_list(x):   
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #print ('names antes if len(names) > 1', names)
        tam = len(names)
        x = []
        if (tam > 1):
            t = 0
            #print ('entrei if')
            while t < tam:
                y = names[t]
                #print ('y', y)
                x.append(y)
                #print ('x', x)
                x.sort()
                #print ('x ordenado', x)
                t = t + 1
        if (tam == 1):            
            x.append(names[0])
            #print ('Unico genero', x)
        return x
        #print ('lista ordenada',x)
    return []
df['generos_ordenados'] = df['genero'].apply(get_list) #O retorno será uma lista com todos os gêneros.

# VALIDANDO TRANSFORMAÇÃO
df[["generos_ordenados", "genero", "titulo_original"]].head(3)

In [ ]:
df.columns

In [ ]:
df.generos_ordenados.value_counts()

In [ ]:
# validando a transformação -> exemplos: "Thirst for Love" / "The Young and the Dead" /  "The Juche Idea"
df[['titulo','generos_ordenados']][df.titulo == 'The Young and the Dead']

In [ ]:
'''
# Identificando e atualizando os filmes que não possuem gênero pela moda ( [] = 2442)
total_registros = len(df)
print ('total de registros', total_registros)
t = z = 0 
while t < total_registros:
    if (len(df['generos_ordenados'][t]) == 0):
        df.genero[t] = df.genero[t].fillna('Drama')
        z = z + 1
    t = t + 1
print ("total de registros sem gênero excluídos", z)
'''

In [ ]:
print ('Qtde de gêneros únicos: ', len(df.generos_ordenados.value_counts()))

In [ ]:
# Identificando os valores únicos dos gênero (da lista de gêneros)
df.generos_ordenados.value_counts().head(10)

In [ ]:
# plotando os generos
df.generos_ordenados.value_counts().head(10).plot.bar(color='green')

plt.xticks(rotation=75)
plt.legend()

plt.xlabel("Lista de gêneros")
plt.ylabel("Média de votos")
plt.title("Gêneros x média de votos")

### Abaixo, transformo os gêneros em features

In [ ]:
# resetando os índices
df = df.reset_index()

In [ ]:
df.columns

In [ ]:
# Criar uma lista que contenha somente valores únicos de gênero, para idêntificá-los e contabilizá-los
todos_registros = len(df.generos_ordenados)        
generos_unicos = [] # lista onde será incluídos os gêneros únicos
x  = 0 
#print ('total de registros do dataset: ', todos_registros)
while (x < todos_registros): # enquanto tiver registros no dataset
    #print ("x:", x)
    generos = df.generos_ordenados[x]
    #print ("P/ x = {} todos os generos são {}:".format(x,generos))
    tam_generos = len(generos)
    #print ("tam_generos: ", tam_generos)
    i = z = 0
    while (i < tam_generos): # enquanto o registro tiver generos
        cada_genero_registo = df.generos_ordenados[x][i]
        #print ("Para x = {} e i = {} os genero é {} ". format(x, i ,cada_genero_registo))
        if (not cada_genero_registo in generos_unicos): # verifica se o genero está na lista de generos únicos
            generos_unicos.append(cada_genero_registo) # inclui o genero na lista de generos únicos
            #print ("lista com os generos únicos", generos_unicos)
            generos_unicos.sort()
            #print ("lista com os generos únicos ordenado", generos_unicos)
            z +=1
        i += 1        
    x += 1


print ("generos únicos: ", len(generos_unicos))
print (generos_unicos)

In [ ]:
# transformar as opções possíveis de genero em features com conteúdo true ou false 
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
te = TransactionEncoder()
genero_x = te.fit(df.generos_ordenados).transform(df.generos_ordenados)
df_novo = pd.DataFrame(genero_x, columns=te.columns_)

In [ ]:
# resetando os índices
df_novo = df_novo.reset_index()

# Mesclado os dois dataframes
horizontal_stack = pd.concat([df, df_novo], axis=1)

In [ ]:
# copiando o conteúdo do novo dataframe para o filme utilizado
df = horizontal_stack.copy()

In [ ]:
# após a transformação do conteúdo da feature genero para novas features, o data set ficou com as seguintes colunas: 
df.columns

In [ ]:
# renomear as colunas de genero e usar todas com caixa baixa para padronizar
df = df.rename(columns={'Action': "acao", 'Adventure': 'aventura', 'Animation': 'animacao', 'Comedy': 'comedia', 'Crime': 'crime', 'Documentary': 'documentario', 'Drama': 'drama', 'Family': 'familia', 'Fantasy': 'fantasia', 'Foreign': 'estrangeiro', 'History': 'historia', 'Horror': 'horror', 'Music': 'musical', 'Mystery': 'misterio', 'Romance': 'romance', 'Science Fiction': 'ficcao_cientifica', 'TV Movie': 'filmes_tv', 'Thriller': 'suspense', 'War': 'gerra', 'Western': 'ocidental'})

In [ ]:
# validando a transformação dos filmes com genero-> exemplos: "Thirst for Love" / "The Young and the Dead" /  "The Juche Idea"
df[['titulo','generos_ordenados','aventura', 'familia', 'fantasia', 'acao', 'misterio']][df.titulo == 'Jumanji']

In [ ]:
# validando a transformação dos filmes sem genero-> exemplos: "Thirst for Love" / "The Young and the Dead" /  "The Juche Idea"
df[['titulo','generos_ordenados','animacao', 'comedia', 'crime']][df.titulo == 'Thirst for Love']

In [ ]:
df[['titulo','generos_ordenados','animacao', 'comedia', 'crime']][df.titulo == "The Young and the Dead"]

In [ ]:
df[['titulo','generos_ordenados','animacao', 'comedia', 'crime']][df.titulo == "The Juche Idea"]

In [ ]:
# contabilizando a qtde de registros true de cada gênero
lista_generos = ['acao', 'aventura', 'animacao', 'comedia', 'crime', 'documentario', 'drama', 'familia', 'fantasia', 'estrangeiro', 'historia', 'horror', 'musical', 'misterio', 'romance', 'ficcao_cientifica', 'filmes_tv', 'suspense', 'gerra', 'ocidental']

tam = len(lista_generos)
print ("total de generos", tam)
x = 0
lista_totais = [] # lista para armazenar os valores totais de cada genero
#print (type (lista_totais))

while x < tam:
    genero = lista_generos[x]
    total_genero = sum(df[genero] == True)    
    print ('genero: ', genero, '|',total_genero)
    x += 1
    lista_totais.append (total_genero)

In [ ]:
#plotando os totais por generos
plt.bar(lista_generos, lista_totais,color = 'black')

plt.rcParams['figure.figsize'] = [15, 10]

plt.xticks(rotation=85)
plt.legend()

plt.xlabel("Genero")
plt.ylabel("Total")
plt.title("Total por gênero")

plt.show()

Conforme observamos, tanto na lista de gêneros únicos e os gêneros separadamente, o mais frequente é o drama, ou seja, a maior parte dos filmes disponível na base tem classificação "Drama" 

## FEATURE LINGUA ORIGINAL

In [ ]:
df.lingua_original.head(3)

In [ ]:
df.lingua_original.describe()

In [ ]:
df.lingua_original.unique()

### Identificando registros nulos e nan de lingua original

In [ ]:
# identificando a qtde de registros com lingua_original = nan
len(df[df.lingua_original.isna()])

In [ ]:
# identificando os registros com lingua_original = nan
df[['titulo','lingua_original']][df.lingua_original.isna()]

CONVERTENDO A SIGLA DO IDIOMA NO NOME DO IDIOMA

In [ ]:
# Convertando a sigla do idioma no nome do idioma

sigla_lingua = df.lingua_original.sort_values().unique()
sigla_lingua

In [ ]:
# Convertando a sigla do idioma no nome do idioma, usando o padrão ISO 639 (https://omegat.sourceforge.io/manual-latest/pt_BR/appendix.languages.html). 

df['lingua'] = df['lingua_original'].map({
    '': 'No Language', ' ': 'No Language',
    'CZ': 'Tcheco',
    'xx': 'No Language',    'en': 'Inglês',     'fr': 'Francês', 
    'zh': 'Chines',    'it': 'Italiano' ,     'fa': 'Persa', 
    'nl': 'Neerlandês',     'de': 'Alemão',     'cn': 'Mandarim padrão', 
    'ar': 'Argentino',     'es' : 'Espanhol',     'ru': 'Russo',
    'sv': 'Espanhol',     'ja': 'Japonês',     'ko': 'Coreano', 
    'sr': 'Sérvio',     'bn': 'Bengali',     'he': 'Hebraico', 
    'pt': 'Português',     'wo': 'Wolof',     'ro': 'Romeno', 
    'cy': 'Galês',     'vi': 'Vietnamita',       'cs':'hrv' , 
       'da': 'Dinamarquês',        'no': 'Norueguês',        'nb': 'Bokmål norueguês', 
       'pl': 'Polonês',        'el': 'Grego, Moderno',        'sh': 'Shan', 
       'mk': 'Macedônio',        'bo': 'Tibetano',        'ca': 'Catalão',
       'fi': 'Finlandês',        'th': 'Tailandês',        'sk': 'Eslovaco', 
       'hu': 'Húngaro',        'bs': 'Bósnio',        'hi': 'Hindi', 
       'tr': 'Turco',        'is': 'Islandês',        'ps': 'Pachto', 
       'ab': 'Abecásio',        'eo': 'Esperanto',       'ka': 'Georgiano', 
       'mn': 'Mongol',        'bm': 'Bambara',        'zu': 'Zulu', 
       'uk': 'Ucraniano',        'af': 'Africâner',        'la': 'Latim', 
       'et': 'Estoniano',        'ku': 'Curdo',        'fy': 'Frísio ocidental', 
       'lv': 'Letão',       'ta': 'Tâmil',        'sl': 'Esloveno', 
       'tl': 'Tagalog',        'ur': 'Urdu',        'rw': 'Kinyarwanda', 
       'id': 'Indonésio',        'bg': 'Búlgaro', 
       'mr': 'Marathi (Marāṭhī)', 'lt': 'Lituano', 'kk': 'Cazaque', 'ms': 'Malaio',
       'sq': 'Albanês', 
       'qu': 'Quechua', 'te': 'Telugu', 'am': 'Amárico', 
       'jv': 'Javanês', 'tg': 'Tajique', 'ml': 'Malayalam', 
       'hr': 'Croata', 'lo': 'Lao', 'ay': 'Aymará',
       'kn': 'Canarês', 
       'eu': 'Basco', 'ne': 'Nepali', 'pa': 'Panjabi, Punjabi', 
       'ky': 'Quirguiz, Kyrgyz', 'gl': 'Galego', 'uz': 'Uzbeque', 
       'sm': 'Samoano', 'mt': 'Maltês', 'hy': 'Armênio', 
       'iu': 'Inuktitut', 'lb': 'Luxemburguês, Letzeburgesch', 'si': 'Sinhala, Cingalês'
       })

In [ ]:
# validando a transformação
sigla_lingua = df.lingua.sort_values().unique()
sigla_lingua

In [ ]:
# validando a transformação
df[['lingua','lingua_original','titulo']][27:32]

In [ ]:
# visualização de dados nan
# validando alterações na feature lingua_original
df[df['lingua'].isnull()]

In [ ]:
# verificando a quantos e quais são os registro "No language"
df[df['lingua'] == 'No Language']

In [ ]:
print ('Qtde de registros:', len(df.lingua.unique()))

In [ ]:
#df.lingua.value_counts().sort_values(ascending = False).head(10)
linguas = df.groupby(by='lingua').size().sort_values(ascending = False)
linguas

In [ ]:
#df.lingua.value_counts().sort_values(ascending = False).plot(color = "red")
linguas.head(20).plot(kind='bar', color = "red")


plt.rcParams['figure.figsize'] = [15, 10]

plt.xticks(rotation=80)

plt.xlabel("Idioma")
plt.ylabel("Total")
plt.title("Total por idioma falado - os 15 mais frequentes")

### FEATURE POPULARIDADE

In [ ]:
df.popularidade.head(3)

In [ ]:
df.popularidade.describe()

In [ ]:
df.popularidade.unique()

In [ ]:
print ('Total de registros únicos de populariade:', len(df.popularidade.unique()))

In [ ]:
# convertendo a featue para float
df.popularidade = df.popularidade.astype('float64').round(3)
df.popularidade.head(15)

In [ ]:
# visualizando os dados agrupados
df.popularidade.value_counts().sort_values(ascending = False)

In [ ]:
df.popularidade.value_counts().sort_values(ascending = False).head(20).plot.bar(color = "blue")

plt.rcParams['figure.figsize'] = [15, 10]

plt.xticks(rotation=90)

plt.xlabel("Popularidade")
plt.ylabel("Total")
plt.title("Total por popularidade - os 10 mais frequentes")

In [ ]:
#Como a feature possui valores muitos discrepantes e não consegui entender qual é o seu range, não irei utilizá-la nas análises do trabalho
df.drop(['popularidade'], inplace=True, axis=1)
df.drop(['index'], inplace=True, axis=1)

# FEATURE COMPANHIA DE PRODUÇÃO (CIA_PRODUCAO)

In [ ]:
df.cia_producao.head(3)

In [ ]:
df.cia_producao.describe()

In [ ]:
print ('Qtde de registros sem cia_producao: ', df.cia_producao.isnull().sum())

In [ ]:
df.cia_producao.describe()

In [ ]:
# Função para transformar a feature em uma lista de generos ordenados
from ast import literal_eval
df['cia_producao'] = df['cia_producao'].apply(literal_eval) # verifica se todos os dados são do mesmo tipo
#Função para pegar os nomes dos gêneros, caso exista.
def get_list(x):   
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #print ('names antes if len(names) > 1', names)
        tam = len(names)
        x = []
        if (tam > 1):
            t = 0
            #print ('entrei if')
            while t < tam:
                y = names[t]
                #print ('y', y)
                x.append(y)
                #print ('x', x)
                x.sort()
                #print ('x ordenado', x)
                t = t + 1
        if (tam == 1):            
            x.append(names[0])
            #print ('Unico genero', x)
        if (tam == 0):            
            x.append('No CIA')
            #print ('Unico genero', x)
        return x
        #print ('lista ordenada',x)
    return []
df['novo_cia_producao'] = df['cia_producao'].apply(get_list) #O retorno será uma lista com todos os gêneros.

# VALIDANDO TRANSFORMAÇÃO
df[["novo_cia_producao", "cia_producao", "titulo_original"]].head(3)

In [ ]:
'''
# transformar a feature em uma nova coluna contento a lista de companhia de cada filme
# Recupera os dados da feature e salva numa nova coluna
from ast import literal_eval
df['cia_producao'] = df['cia_producao'].apply(literal_eval) 
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 3:
            names = names[:3]
        return names
    return []
df['novo_cia_producao'] = df['cia_producao'].apply(get_list) 

# VALIDANDO TRANSFORMAÇÃO
df[["novo_cia_producao", "cia_producao", "titulo_original"]].head(3)
'''

In [ ]:
print ('Qtde de registros unicos de CIA de produção: ', len(df.novo_cia_producao.value_counts()))

In [ ]:
# As 10 CIA mais frequentes
cia_10_mais = df.novo_cia_producao.value_counts().sort_values(ascending = False).head(12)
cia_10_mais

In [ ]:
# Plotando as 10 cia de produção mais frequentes
cia_10_mais.plot.bar(color = "pink")

plt.rcParams['figure.figsize'] = [15, 10]

plt.xticks(rotation=90)

plt.xlabel("CIA de produção")
plt.ylabel("Total")
plt.title("Total por CIA - os 10 mais frequentes")

### TRANSFORMANDO AS 10 CIA MAIS FREQUENTES EM FEATURES

In [ ]:
# transformar as opções possíveis de genero em features com conteúdo true ou false 
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
te = TransactionEncoder()
cia_x = te.fit(df.novo_cia_producao).transform(df.novo_cia_producao)
df_cia = pd.DataFrame(cia_x, columns=te.columns_)

In [ ]:
# SELECIONANDO AS 10 MAIS FREQUENTES
df_cia = df_cia[['Metro-Goldwyn-Mayer (MGM)', 'Warner Bros.', 'Paramount Pictures', 
        'Twentieth Century Fox Film Corporation', 'Universal Pictures', 'RKO Radio Pictures',
        'Columbia Pictures','Mosfilm', 'Walt Disney Pictures','Universal International Pictures (UI)']]

In [ ]:
# Mesclado os dois dataframes
horizontal_stack = pd.concat([df, df_cia], axis=1)

# copiando o conteúdo do novo dataframe para o filme utilizado
df = horizontal_stack.copy()

In [ ]:
print ('Qtde de colunas após mesclagem:', df.shape)

In [ ]:
# validando a transformação
# exemplo: filme: [Dracula: Dead and Loving It] => CIA: ['Columbia Pictures', 'Castle Rock Entertainment', 'Enigma Pictures']
# exemplo: filme: [The Indian in the Cupboard] => CIA: [['Paramount Pictures', 'Columbia Pictures Corporation', 'The Kennedy/Marshall Company']

df[['Metro-Goldwyn-Mayer (MGM)', 'Warner Bros.', 'Paramount Pictures', 
        'Twentieth Century Fox Film Corporation', 'Universal Pictures', 'RKO Radio Pictures', 'Columbia Pictures',
        'Mosfilm', 'Walt Disney Pictures','Universal International Pictures (UI)']][(df.titulo == 'Dracula: Dead and Loving It') | (df.titulo == 'The Indian in the Cupboard')]


In [ ]:
# Features até agora
df.columns

### FEATURE PAIS DE PRODUÇÃO

In [ ]:
df['pais_producao'].head(3)

In [ ]:
df['pais_producao'].isnull().sum()

In [ ]:
# Função para transformar dicionario em uma lista de pais
df['pais_producao'] = df['pais_producao'].apply(literal_eval) # verifica se todos os dados são do mesmo tipo
def get_list(x):   
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #print ('names antes if len(names) > 1', names)
        tam = len(names)
        x = []
        if (tam > 1):
            t = 0
            #print ('entrei if')
            while t < tam:
                y = names[t]
                #print ('y', y)
                x.append(y)
                #print ('x', x)
                x.sort()
                #print ('x ordenado', x)
                t = t + 1
        if (tam == 1):            
            x.append(names[0])
            #print ('Unico genero', x)
        return x
        #print ('lista ordenada',x)
    return []
df['pais_producao_ordenados'] = df['pais_producao'].apply(get_list) #O retorno será uma lista com todos os gêneros.

# VALIDANDO TRANSFORMAÇÃO
df[["pais_producao_ordenados", "pais_producao", "titulo_original"]].head(3)

In [ ]:
# lista dos 10 países mais comuns
pais_15_mais = df.pais_producao_ordenados.value_counts().sort_values(ascending = False).head(15)
pais_15_mais

In [ ]:
pais_15_mais.plot.bar(color = "red")

plt.rcParams['figure.figsize'] = [15, 10]

plt.xticks(rotation=90)

plt.xlabel("País de produção")
plt.ylabel("Total")
plt.title("Total por paós - os 10 mais frequentes")

Comparar a lista de países com os países convertidos para feature

In [ ]:
'''# Criar uma lista que contenha somente valores únicos do pais de produção, para idêntificá-los e contabilizá-los
todos_registros = len(df.novo_pais_producao)        
pais_producao_unicos = [] # lista onde será incluídos os países de produção únicos
x = 0 
print ('total de registros do dataset: ', todos_registros)
while (x < todos_registros): # enquanto tiver registros no dataset
    pais = df.novo_pais_producao[x]
    tam_pais = len(pais)
    i = z = 0
    while (i < tam_pais): # enquanto o registro tiver paises de produção
        cada_pais_registo = df.novo_pais_producao[x][i]
        if (not cada_pais_registo in pais_producao_unicos): # verifica se o pais de produção está na lista de paises únicos
            pais_producao_unicos.append(cada_pais_registo) # inclui o pais na lista de paises únicos
            z +=1
        i += 1        
    x += 1

print ("pais_producao_unicos: ", len(pais_producao_unicos))
print ('Lista de país onde os filmes foram produzidos: \n', pais_producao_unicos)
'''

In [ ]:
df.pais_producao_ordenados.value_counts().head(15)

In [ ]:
# transformar as opções possíveis de pais de gravação em features com conteúdo true ou false 
pais_producao_x = te.fit(df.pais_producao_ordenados).transform(df.pais_producao_ordenados)
df_novo_pais_producao = pd.DataFrame(pais_producao_x, columns=te.columns_)

In [ ]:
# resetando os índices
df_novo_pais_producao = df_novo_pais_producao.reset_index()
df_novo_pais_producao.head(3)

In [ ]:
# SELECIONANDO AS 10 MAIS FREQUENTES
df_novo_pais_producao = df_novo_pais_producao[['United States of America','United Kingdom','France','Japan','Italy','Canada','Germany','Russia','India', 'Canada']]
df_novo_pais_producao.head(3)

In [ ]:
# renomear as coluna para deferenciar do pais dos idiomas (produção)
df_novo_pais_producao = df_novo_pais_producao.rename(columns={'United States of America': "United States of America (pais producao)", 'United Kingdom': 'United Kingdom (pais producao)', 'France': 'France (pais producao)', 'Japan': 'Japan (pais producao)', 'Italy': 'Italy (pais producao)', 'Canada': 'Canada (pais producao)', 'Germany': 'Germany (pais producao)', 'Russia': 'Russia (pais producao)', 'India': 'India (pais producao)', 'Canada': 'Canada (pais producao)'})

In [ ]:
df_novo_pais_producao.shape

In [ ]:
df.shape

In [ ]:
# Mesclado os dois dataframes
horizontal_stack = pd.concat([df, df_novo_pais_producao], axis=1)

# copiando o conteúdo do novo dataframe
df = horizontal_stack.copy()

In [ ]:
print ('Qtde de colunas após mesclagem:', df.shape)

In [ ]:
df.columns

In [ ]:
# validando a transformação
# exemplo: filme: [Dracula: Dead and Loving It] => paises: ['France','United States of America'] -> id: 11
# exemplo: filme: [GoldenEye] => paises ['United Kingdom', 'United States of America']-> id => 9


df[['United States of America (pais producao)','United Kingdom (pais producao)','France (pais producao)','Japan (pais producao)','Italy (pais producao)','Canada (pais producao)','Germany (pais producao)','Russia (pais producao)','India (pais producao)', 'Canada (pais producao)']][(df.titulo == 'Dracula: Dead and Loving It') | (df.titulo == 'GoldenEye')]

In [ ]:
'''
# contabilizando a qtde de registros true de cada pais de gravação
lista_pais = df_novo_pais_producao.columns
print (lista_pais)
tam = len(lista_pais)
print ("total de pais", tam)
x = 0
lista_totais = [] # lista para armazenar os valores totais de cada genero
#print (type (lista_totais))

while x < tam:
    pais = lista_pais[x]
    total_pais = sum(df_novo_pais_producao[pais] == True)    
    print ('pais: ', pais, '|',total_pais)
    x += 1
    lista_totais.append (total_pais)

lista_pais_producao = list(zip(lista_totais,lista_pais))

del(lista_pais_producao[0]) # excluindo o valor index

print (lista_pais_producao)
'''

In [ ]:
'''
# ordenando e revertendo a ordenação
lista_pais_producao.sort(reverse=True)

# obtendo os 10 mais frequentes países
lista_pais_producao[0:10]
'''

In [ ]:
'''
#plotando os 10 mais frequentes paises de gravação
x = [v[1] for v in lista_pais_producao[0:10]]
y = [v[0] for v in lista_pais_producao[0:10]]

plt.bar(x, y)

plt.rcParams['figure.figsize'] = [15, 5]
plt.ylim(0, 25000)

plt.xticks(rotation=75)

plt.xlabel("País de gravação")
plt.ylabel("Total")
plt.title("Total por país onde os filmes foram gravados")

plt.show()
'''

## FEATURE DATA DE LANÇAMENTO

In [ ]:
df.data_lancamento.head(3)

In [ ]:
df.data_lancamento.describe()

In [ ]:
# Separa o ano da data de lançamento e salvar numa nova featue
data = df['data_lancamento'].astype(str) # se o resultado for uma lista das datas do banco
print (data)
ano = [a.split('-')[0] for a in data]
df['ano_lancamento'] = ano

# validando a transformação
df[['data_lancamento','ano_lancamento']].head(2)

In [ ]:
df.ano_lancamento.unique()

In [ ]:
# obtendo o valor mais frequente (a moda não funcionou)
df.ano_lancamento.value_counts(ascending = False)

In [ ]:
ano_lancamento = df.groupby(by='ano_lancamento').size().sort_values(ascending = False).head(10)
ano_lancamento

In [ ]:
# visualizando os 10 anos mais frequentes
ano_lancamento.plot.bar(color = 'gray')

plt.rcParams['figure.figsize'] = [15, 5]
plt.ylim(0, 2100)

plt.xticks(rotation=0)

plt.ylabel('Qtde')
plt.xlabel('Ano de lançamentos')
plt.title('Os 10 anos de lançamento mais frequentes')

In [ ]:
## FEATURE RECEITA

In [ ]:
df.receita.head(3)

In [ ]:
df.receita.describe()

In [ ]:
# qtde de registros únicos de receita
len(df.receita.unique())

Conforme observado, 75% dos filmes não possui valor de receita, portanto, a feature não será utilizada na análise/predição

In [ ]:
#Deletando
df.drop('receita', axis=1, inplace = True)

## FEATURE DURACAO

In [ ]:
df.duracao.head(3)


In [ ]:
df.duracao.describe()

In [ ]:
# qtde de registros únicos
len(df.duracao.unique())

In [ ]:
# As 10 duração mais frequentes
df.groupby(by='duracao').size().sort_values(ascending = False).head(10)

In [ ]:
# média
df['duracao'].mean().round(0)

In [ ]:
# muitos registros possuem duração zero. Estes casos serão substituidos pela média
df.duracao= df.duracao.replace(0,df['duracao'].mean().round(0))

In [ ]:
# verificando o impacto da atualização dos valores zerados de duração
df.groupby(by='duracao').size().sort_values(ascending = False).head(10)

In [ ]:
# verificando os valores nulos
df[df.duracao.isnull()]

In [ ]:
# atualizando os dados faltantes
df.duracao = df.duracao.fillna(df['duracao'].mean().round(0))

In [ ]:
# verificando a atualização
df[df.duracao.isnull()]

In [ ]:
df.groupby(by='duracao').size().sort_values(ascending = False).head(10).plot.bar()

plt.grid(True)
plt.rcParams['figure.figsize'] = [15, 10]

plt.xticks(rotation=0)

plt.ylabel('Qtde')
plt.xlabel('Duração')
plt.title('Duração mais frequentes')

## FEATURE IDIOMAS FALADOS (TRADUÇÕES)

In [ ]:
df.idiomas_falados.head(3)

In [ ]:
df.idiomas_falados.describe()

In [ ]:
df.idiomas_falados.isnull().sum()

In [ ]:
# identificando os registros nulos
df[df.idiomas_falados.isnull()]

In [ ]:
# Deletando os registro
df.drop(19721, inplace=True, axis=0)
df.drop(29481, inplace=True, axis=0)
df.drop(35561, inplace=True, axis=0)

In [ ]:
# validando a exclusao dos registros nulos
df[df.idiomas_falados.isnull()]

In [ ]:
'''
# Função para transformar dicionario em uma lista de pais
df['idiomas_falados'] = df['idiomas_falados'].apply(literal_eval) # verifica se todos os dados são do mesmo tipo
def get_list(x):   
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #print ('names antes if len(names) > 1', names)
        tam = len(names)
        x = []
        if (tam > 1):
            t = 0
            #print ('entrei if')
            while t < tam:                
                y = names[t]
                #print ('y', y)
                x.append(y)
                #print ('x', x)
                x.sort()
                #print ('x ordenado', x)
                t = t + 1
        if (tam == 1):            
            x.append(names[0])
            #print ('Unico genero', x)
        if (tam == 0):
            x.append('English') # valor calculado através da moda
        return x
        #print ('lista ordenada',x)
    return []
df['idiomas_falados_ordenados'] = df['idiomas_falados'].apply(get_list) #O retorno será uma lista com todos os gêneros.

# VALIDANDO TRANSFORMAÇÃO
df[["idiomas_falados_ordenados", "idiomas_falados", "titulo_original"]].head(3)
'''

In [ ]:
# resetando os índices
'''df = df.reset_index()'''

In [ ]:
# Função para transformar dicionario em uma lista de pais
#                elif cada_idioma_registo == '??????': # equivale a sigla ky = 'Quirguiz, Kyrgyz'
#                elif cada_idioma_registo == '?????': # equivale a sigla 'kn' = 'Canarês'      
# Recupera os dados da feature e salva numa nova coluna
from ast import literal_eval
df['idiomas_falados'] = df['idiomas_falados'].apply(literal_eval) 
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        valores = []
        if len(names) > 0:
            names = names[:3]    
        return names
    return []
#df['novo_idiomas_falados'] = df['idiomas_falados'].apply(get_list) 
df['idiomas_falados_ordenados'] = df['idiomas_falados'].apply(get_list) 

# VALIDANDO TRANSFORMAÇÃO
df[["idiomas_falados_ordenados", "idiomas_falados", "titulo_original"]].head(3)  

In [ ]:
# registros vazios: 83,107,126, 137)
df[["idiomas_falados_ordenados", "idiomas_falados", "titulo_original"]][120:140]

In [ ]:
df.idiomas_falados_ordenados.value_counts()

In [ ]:
'''
# atualizando os registros zerados
total_registros = len(df)
print ('total de registros', total_registros)
t = z = 0 
while t < total_registros:
    if (len(df['idiomas_falados_ordenados'][t]) == 0):
        df['idiomas_falados_ordenados'][t].append(df.lingua_original[t])
        z = z + 1
    t = t + 1
print ("total de registros atualizados", z)
'''

In [ ]:
# Validando se os registros zerados foram atualizados
idiomas = df['idiomas_falados_ordenados'].value_counts().head(10)
idiomas

In [ ]:
idiomas.plot.bar(color='orange')

plt.xticks(rotation=75)

plt.xlabel("Lista de idiomas/traduções")
plt.ylabel("Total")
plt.title("Os 10 idiomas mais frequentes")

In [ ]:
# resetando os índices
df = df.reset_index()

In [ ]:
# Criar uma lista que contenha somente valores únicos de linguas, para idêntificá-los e contabilizá-los
# # existem registros com valores '??????' e '?????' (Turkic), estes NÃO ESTÃO SENDO TRATADOS ????????????????????????????????????????????????????????????????????????????????????????????????????? 
# https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes

todos_registros = len(df.idiomas_falados_ordenados)        
idiomas_falados_unicos = [] # lista onde será incluídos os linguas únicos
x = 0 
print ('total de registros do dataset: ', todos_registros)
while (x < todos_registros): # enquanto tiver registros no dataset
    #print ("x:", x)
    idiomas = df.idiomas_falados_ordenados[x]
    tam_idiomas = len(idiomas)
    i = z = b = 0
    while (i < tam_idiomas): # enquanto o registro tiver linguas
        cada_idioma_registo = df.idiomas_falados_ordenados[x][i]
        if (not cada_idioma_registo in idiomas_falados_unicos): # verifica se a lingua está na lista
            if (cada_idioma_registo != '??????') and (cada_idioma_registo != '?????'):               
                idiomas_falados_unicos.append(cada_idioma_registo) # inclui na lista de linguas unicas
            z +=1
        i += 1        
    x += 1

print ("idiomas_falados_unicos: ", len(idiomas_falados_unicos))
print (idiomas_falados_unicos)

In [ ]:
df.columns

In [ ]:
# transformar as opções possíveis em features com conteúdo true ou false 
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
te = TransactionEncoder()
idiomas_x = te.fit(df.idiomas_falados_ordenados).transform(df.idiomas_falados_ordenados)
df_novo = pd.DataFrame(idiomas_x, columns=te.columns_)

In [ ]:
# selecionando as features que serão mescladas - somente as 5 mais frequentes
# [English], [Français], [日本語], [Italiano], [Español]
df_novo = df_novo[['English', 'Français', '日本語', 'Italiano', 'Español']]
df_novo

In [ ]:
df_novo.columns

In [ ]:
df.columns

In [ ]:
# resetando os índices
df_novo = df_novo.reset_index()

# Mesclado os dois dataframes
horizontal_stack = pd.concat([df, df_novo], axis=1)

# copiando o conteúdo do novo dataframe para o filme utilizado
df = horizontal_stack.copy()

In [ ]:
df.columns

# Features que serão utilizadas no trabalho



In [ ]:
df_reduzido = df[['titulo','generos_ordenados','descricao','duracao', 'status', 'video', 'ano_lancamento',
       'lingua', 'qtde_votos', 
       'acao', 'aventura', 'animacao', 'comedia', 'crime', 'documentario', 'drama', 'familia',
       'fantasia', 'estrangeiro', 'historia', 'horror', 'musical', 'misterio',
       'romance', 'ficcao_cientifica', 'filmes_tv', 'suspense', 'gerra','ocidental', 
       'Metro-Goldwyn-Mayer (MGM)', 'Warner Bros.', 'Paramount Pictures',
       'Twentieth Century Fox Film Corporation', 'Universal Pictures',
       'RKO Radio Pictures', 'Columbia Pictures', 'Mosfilm', 'Walt Disney Pictures', 'Universal International Pictures (UI)',
       'United States of America (pais producao)', 'United Kingdom (pais producao)','France (pais producao)', 'Japan (pais producao)', 'Italy (pais producao)', 'Canada (pais producao)', 'Germany (pais producao)', 'Russia (pais producao)', 'India (pais producao)', 'Canada (pais producao)', 
       'English', 'Français', '日本語','Italiano', 'Español', 
       'media_votos']]

In [ ]:
print (" Tamanho original:", filmes.shape[0], "| tamanho final: ",  df_reduzido.shape[0], "| Redução de:", filmes.shape[0] - df_reduzido.shape[0], "registros.")

In [ ]:
print ("Qtde Features original:", filmes.shape[1], "| Qtde Features final: ",  df_reduzido.shape[1], "| Aumento de:", df_reduzido.shape[1] - filmes.shape[1], "registros.")

# identificando OUTLIERS

## duracao x media_votos

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(df_reduzido.duracao, df_reduzido.media_votos)

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(x=df_reduzido.duracao, y=df_reduzido.media_votos)
plt.xlabel("Duração", fontsize=13)
plt.ylabel("Média Votos", fontsize=13)
plt.ylim(0,10)

In [ ]:
# idenficando os registros acima de 1000 horas
# os filmes com muitos minutos são mini-séries e séries, portanto não foram classificados como outlier
df_reduzido[(df_reduzido["duracao"]>1000)&(df_reduzido["media_votos"]<9)&(df_reduzido["media_votos"]>4)]

In [ ]:
# idenficando os registros com duração abaixo de 10
# o filmes com duranção quase zero são filmes de curta metragem, portanto, não são outliers
df_reduzido[(df_reduzido["duracao"]<10)&(df_reduzido["media_votos"]<1)].head(10)

In [ ]:
plt.rc('figure', figsize = (14, 6)) # definindo o tamanho da figura
df_reduzido.boxplot(['duracao'])

In [ ]:
# definindo os limites do boxplot
Q1 = df_reduzido['duracao'].quantile(.25)
Q3 = df_reduzido['duracao'].quantile(.75)
IIQ = Q3 - Q1
limite_inferior = Q1 - 1.5 * IIQ
limite_superior = Q3 + 1.5 * IIQ

In [ ]:
print ('Q1 =', Q1, '| Q3=', Q3,  '| IIQ=', IIQ, '| lim-inf=', limite_inferior, '| lim_sup=', limite_superior)

In [ ]:
# selecionando os registros dentro dos limites (considerados não outlier)
selecao = (df_reduzido['duracao'] >= limite_inferior) & (df_reduzido['duracao'] <= limite_superior)
dados_new = df_reduzido[selecao]
dados_new[['titulo','generos_ordenados', 'ano_lancamento', 'qtde_votos','media_votos']]

In [ ]:
# Plotanto os dados sem o outlier
dados_new.boxplot(['duracao'])

In [ ]:
# comparando a quantidade de registros
diferenca = df_reduzido.shape[0] - df_novo.shape[0]
print('Dados com outliers:', df_reduzido.shape[0], "Dados sem outliers: ", df_novo.shape[0], " Diferença:", diferenca)

Baseado na feature duração, não houve filme considerado como outlier.

## status x media_votos

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(df_reduzido.status, df_reduzido.media_votos)

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(x=df_reduzido.status, y=df_reduzido.media_votos)
plt.xlabel("Status", fontsize=13)
plt.ylabel("Média Votos", fontsize=13)
plt.ylim(0,10)

In [ ]:
# filmes que não possuem votos e portanto nem média de votos. Serão excluídos
df_reduzido[(df_reduzido.media_votos ==0) & (df_reduzido.qtde_votos == 0)]

In [ ]:
# exclusão dos registros sem votos e sem média de votos
df_reduzido.drop(df_reduzido[(df_reduzido.media_votos ==0) & (df_reduzido.qtde_votos == 0)].index,inplace=True)

In [ ]:
# identificando os filmes que estão no limite superior e inferior do status = released
# este filmes tiveram de 1 a 3 votos e receberam média altas, logo não foram descartados
df_reduzido[(df_reduzido["status"]=='Released') & (df_reduzido["media_votos"]>9.8)]

In [ ]:
df_reduzido[(df_reduzido["status"]=='Released') & (df_reduzido["media_votos"]>9.8)].sort_values(by='qtde_votos')

In [ ]:
df_reduzido.status.value_counts()

In [ ]:
# identificando os filmes que não foram lançados (released) para avaliar a qtde de votos e média de votos
df_reduzido[(df_reduzido["status"]=='Rumored') | (df_reduzido["status"]=='Post Production') 
        | (df_reduzido["status"]=='Canceled') | (df_reduzido["status"]=='In Production')
        | (df_reduzido["status"]=='Planned')].sort_values(by=['status','qtde_votos','media_votos'])

In [ ]:
# estes filmes não foram lançados na época de difponibilização deste dataset, portanto, optei por excluir estes filmes e deletar a feature status
df_reduzido.drop(df_reduzido[(df_reduzido["status"]=='Rumored') | (df_reduzido["status"]=='Post Production') 
        | (df_reduzido["status"]=='Canceled') | (df_reduzido["status"]=='In Production')
        | (df_reduzido["status"]=='Planned')].index, inplace=True)

In [ ]:
# validando a exclusão
df_reduzido[(df_reduzido["status"]=='Rumored') | (df_reduzido["status"]=='Post Production') 
        | (df_reduzido["status"]=='Canceled') | (df_reduzido["status"]=='In Production')
        | (df_reduzido["status"]=='Planned')].sort_values(by=['status','qtde_votos','media_votos'])

In [ ]:
# deletando a feature status
df_reduzido.drop(['status'],axis = 1, inplace = True)

## qtde_votos x media_votos 

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(df_reduzido.qtde_votos, df_reduzido.media_votos)

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(x=df_reduzido.qtde_votos, y=df_reduzido.media_votos)
plt.xlabel("Qtde de votos", fontsize=13)
plt.ylabel("Média Votos", fontsize=13)
plt.ylim(0,10)

### outra forma de analisar OUTLIERS

In [ ]:
df_reduzido.boxplot(['qtde_votos'])

In [ ]:
Q1 = df_reduzido['qtde_votos'].quantile(.25)
Q3 = df_reduzido['qtde_votos'].quantile(.75)
IIQ = Q3 - Q1
limite_inferior = Q1 - 1.5 * IIQ
limite_superior = Q3 + 1.5 * IIQ

In [ ]:
print ('Q1 =', Q1, '| Q3=', Q3,  '| IIQ=', IIQ, '| lim-inf=', limite_inferior, '| lim_sup=', limite_superior)

In [ ]:
# visualizando os registros que estão entre o limite inferior e o superior
selecao = (df_reduzido['qtde_votos'] >= limite_inferior) & (df_reduzido['qtde_votos'] <= limite_superior)
dados_new = df_reduzido[selecao]
dados_new[['titulo', 'duracao','ano_lancamento','qtde_votos','media_votos']]

In [ ]:
# visualizando os registros que estão abaixo do limite inferior
selecao = (df_reduzido['qtde_votos'] < limite_inferior)
dados_abaixo_inferior = df_reduzido[selecao]
dados_abaixo_inferior[['titulo', 'duracao','ano_lancamento','qtde_votos','media_votos']]

In [ ]:
# visualizando os registros que estão acima do limite superior
selecao = (df_reduzido['qtde_votos'] > limite_superior)
dados_acima_superior = df_reduzido[selecao]
dados_acima_superior[['titulo', 'duracao','ano_lancamento','qtde_votos','media_votos']]

In [ ]:
# comparando a quantidade de registros
diferenca = df_reduzido.shape[0] - df_novo.shape[0]
print('Dados com outliers:', df_reduzido.shape[0], "Dados sem outliers: ", df_novo.shape[0], " Diferença:", diferenca)

Analisei os filmes e não achei que os filmes são outliers. Alguns são mais velhos, logo possuiem muitos votos e outros tem poucos votos, mas com classificação alto.

# convertendo os tipos das features

In [ ]:
df_reduzido.dtypes

In [ ]:
df.duracao.round(0)

In [ ]:
df_reduzido.video = df_reduzido.video.astype('bool')
df_reduzido.ano_lancamento = df_reduzido.ano_lancamento.astype('int64')

In [ ]:
df_reduzido.dtypes

In [ ]:
df_reduzido.columns

## Salvando em um arquivo

In [ ]:
# salvando o dataset num CSV      
df_reduzido.to_csv("C:/Users/Herica/Desktop/Herica/cursoIACD/bases/filmes/filmes_df_limpo.csv",columns=['titulo','descricao','duracao', 'video', 'ano_lancamento',
       'lingua', 'qtde_votos', 
       'acao', 'aventura', 'animacao', 'comedia', 'crime', 'documentario', 'drama', 'familia',
       'fantasia', 'estrangeiro', 'historia', 'horror', 'musical', 'misterio',
       'romance', 'ficcao_cientifica', 'filmes_tv', 'suspense', 'gerra','ocidental', 
       'Metro-Goldwyn-Mayer (MGM)', 'Warner Bros.', 'Paramount Pictures',
       'Twentieth Century Fox Film Corporation', 'Universal Pictures',
       'RKO Radio Pictures', 'Columbia Pictures', 'Mosfilm', 'Walt Disney Pictures', 'Universal International Pictures (UI)',
       'United States of America (pais producao)',
       'United Kingdom (pais producao)', 'France (pais producao)',
       'Japan (pais producao)', 'Italy (pais producao)',
       'Canada (pais producao)', 'Canada (pais producao)',
       'Germany (pais producao)', 'Russia (pais producao)',
       'India (pais producao)', 'Canada (pais producao)',
       'Canada (pais producao)', 
       'English', 'Français', '日本語','Italiano', 'Español', 
       'media_votos'],index=False)

### 4) **(30%)** Análise

Com seu dado pronto, é hora de fazer a parte mais legal: investigar e responder nossas hipoteses. Elas se confirmam? Em quais cenários? Existe alguma forma de perceber isso no mundo real? Dica: Abuse das técnicas de visualização.

Ex: Ainda no contexto dos exemplos do item 2). Ao investigar a hipotese relacionada ao padrão de compra, percebi que em todos os meses existia uma diferença entre dia util e final de semana. Porém notei que Fevereiro possui um comportamento diferente, o que poderia explicar? Talvez o carnaval e seus feriados.

Ex2: Talvez eu descubra que alguns finais de semana possuam um comportamento diferente dos dias de semana e outros não. Por que não são todos? Nessa investigação você pode acabar descobrindo que os finais de semana que são diferentes, coincidem no fato de serem os primeiros dias utéis, o que pode remeter ao fato de boa parte das empresas realizarem pagamentos nessa parte do mês.

Questionamentos a serem investigados:

1 - As maiores média de votos está relacionada a algum genero específico?

2 - As maiores média de votos está relacionada a algum idioma original?

3 - As maiores média de votos está relacionada a algum cia de produção?

In [ ]:
# lendo o arquivo
df_final = pd.read_csv(r'C:/Users/Herica/Desktop/Herica/cursoIACD/bases/filmes/filmes_df_limpo.csv')

## Hipótese 1 - As maiores média de votos está relacionada a algum genero específico?

In [ ]:
df_final.dtypes

In [ ]:
# Como são várias as features de gênero, as dividi em grupo para facilitar a visualização
x = df_final.groupby(['media_votos']).sum()

x['acao'].plot(kind = 'line',color='green', title='Total da acao/target por V1',grid=True, stacked=True,label='Ação')
x['aventura'].plot(kind = 'line',color='red', title='Total da acao/target por V1',grid=True, stacked=True,label='Aventura')
x['animacao'].plot(kind = 'line',color='blue', title='Total da acao/target por V1',grid=True, stacked=True,label='Animação')
x['comedia'].plot(kind = 'line',color='pink', title='Total da acao/target por V1',grid=True, stacked=True,label='Comédia')
x['crime'].plot(kind = 'line',color='magenta', title='Total da acao/target por V1',grid=True, stacked=True,label='Crime')

plt.legend()
plt.xlabel("Gêneros")
plt.ylabel("Média Votos")


# Neste gráfico, percebemos que a comédia é o gênero com maior média de votos, seguido da ação

In [ ]:
# Como são várias as features de gênero, as divide
x = df_final.groupby(['media_votos']).sum()

x['documentario'].plot(kind = 'line',color='green', title='Total da acao/target por V1',grid=True, stacked=True,label='Documentário')
x['drama'].plot(kind = 'line',color='red', title='Total da acao/target por V1',grid=True, stacked=True,label='Drama')
x['familia'].plot(kind = 'line',color='blue', title='Total da acao/target por V1',grid=True, stacked=True,label='Família')
x['fantasia'].plot(kind = 'line',color='pink', title='Total da acao/target por V1',grid=True, stacked=True,label='Fantasia')
x['estrangeiro'].plot(kind = 'line',color='magenta', title='Total da acao/target por V1',grid=True, stacked=True,label='Estrangeiro')

plt.legend()
plt.xlabel("Gêneros")
plt.ylabel("Média Votos")


# Neste gráfico, percebemos que o drama é o gênero com maior média de votos, estando os demais mais ou menos no memo nível

In [ ]:
# Como são várias as features de gênero, as divide
x = df_final.groupby(['media_votos']).sum()

x['historia'].plot(kind = 'line',color='green', title='Total da acao/target por V1',grid=True, stacked=True,label='História')
x['horror'].plot(kind = 'line',color='red', title='Total da acao/target por V1',grid=True, stacked=True,label='Horror')
x['musical'].plot(kind = 'line',color='blue', title='Total da acao/target por V1',grid=True, stacked=True,label='Musical')
x['misterio'].plot(kind = 'line',color='pink', title='Total da acao/target por V1',grid=True, stacked=True,label='Mistério')
x['romance'].plot(kind = 'line',color='magenta', title='Total da acao/target por V1',grid=True, stacked=True,label='Romance')

plt.legend()
plt.xlabel("Gêneros")
plt.ylabel("Média Votos")


# Neste gráfico, percebemos que o romance é o gênero com maior média de votos, segido do horror.

In [ ]:
# Como são várias as features de gênero, as divide
x = df_final.groupby(['media_votos']).sum()

x['ficcao_cientifica'].plot(kind = 'line',color='green', title='Total da acao/target por V1',grid=True, stacked=True,label='Ficção')
x['filmes_tv'].plot(kind = 'line',color='red', title='Total da acao/target por V1',grid=True, stacked=True,label='Seriados')
x['suspense'].plot(kind = 'line',color='blue', title='Total da acao/target por V1',grid=True, stacked=True,label='Suspense')
x['gerra'].plot(kind = 'line',color='pink', title='Total da acao/target por V1',grid=True, stacked=True,label='Gerra')
x['ocidental'].plot(kind = 'line',color='magenta', title='Total da acao/target por V1',grid=True, stacked=True,label='Ocidental')

plt.legend()
plt.xlabel("Gêneros")
plt.ylabel("Média Votos")


# Neste gráfico, percebemos que o suspense é o gênero com maior média de votos, seguido da ficção

In [ ]:
# plotando os maiores dos gêneros obtidos nos gráficos acima: comédia, suspense, romance e drama

# Como são várias as features de gênero, as divide
x = df_final.groupby(['media_votos']).sum()

x['comedia'].plot(kind = 'line',color='pink',grid=True, stacked=True,label='Comédia')
x['suspense'].plot(kind = 'line',color='blue',grid=True, stacked=True,label='Suspense')
x['romance'].plot(kind = 'line',color='magenta',grid=True, stacked=True,label='Romance')
x['drama'].plot(kind = 'line',color='red', grid=True, stacked=True,label='Drama')

plt.legend()
plt.title('GÊneros x Média Votos')
plt.xlabel("Gêneros")
plt.ylabel("Média Votos")


# Neste gráfico, percebemos que o drama é o gênero com maior média de votos, seguido da comédia. Os demais estão parecidos.

Conforme observamos, o gênero que possui a maior média de votos é o Drama, seja isolado, seja em conjunto com outros gêneros. Dos 10 maiores, o Drama está presente em 6. 

## Hipótese 2 - As maiores média de votos está relacionada a algum idioma original?

In [ ]:
df_final.columns

In [ ]:
lingua = df_final[['lingua','media_votos']].groupby('lingua').sum().sort_values(by = 'media_votos', ascending = False).head(10)
lingua

In [ ]:
# plotando os 10 idiomas originais com maior média de votos
x = df_final[['lingua','media_votos']].groupby('lingua').sum().sort_values(by = 'media_votos', ascending = False).head(10)
x = x.reset_index()

plt.bar(x.lingua,x.media_votos,color='red')


plt.xticks(rotation=75)
plt.xlabel("Lingua original")
plt.ylabel("Média de votos")
plt.title("Lingua x média de votos")

In [ ]:
total_ingles = len(df_final[df_final['lingua'] == "Inglês"])

outras_linguas = ['Francês', 'Chines', 'Italiano', 'Persa', 'Neerlandês',
       'Alemão', 'Mandarim padrão', 'Argentino', 'Espanhol', 'Russo',
       'Japonês', 'Coreano', 'Sérvio', 'Bengali', 'Hebraico', 'Português',
       'Wolof', 'Romeno', 'Húngaro', 'Galês', 'Vietnamita', 'hrv',
       'Dinamarquês', 'Norueguês', 'Bokmål norueguês', 'Polonês',
       'Grego, Moderno', 'Shan', 'No Language', 'Macedônio', 'Tibetano',
       'Catalão', 'Finlandês', 'Tailandês', 'Eslovaco', 'Bósnio', 'Hindi',
       'Turco', 'Islandês', 'Pachto', 'Abecásio', 'Esperanto',
       'Georgiano', 'Mongol', 'Bambara', 'Zulu', 'Ucraniano', 'Africâner',
       'Latim', 'Estoniano', 'Curdo', 'Letão', 'Tâmil', 'Esloveno',
       'Tagalog', 'Urdu', 'Kinyarwanda', 'Indonésio', 'Búlgaro',
       'Marathi (Marāṭhī)', 'Lituano', 'Cazaque', 'Malaio', 'Albanês',
       'Quechua', 'Telugu', 'Amárico', 'Javanês', 'Tajique', 'Malayalam',
       'Croata', 'Lao', 'Aymará', 'Canarês', 'Nepali', 'Panjabi, Punjabi',
       'Galego', 'Quirguiz, Kyrgyz', 'Samoano', 'Basco', 'Tcheco',
       'Armênio', 'Inuktitut', 'Sinhala, Cingalês']
total_outras = 0
for i in outras_linguas:
    total_outras = total_outras + len(df_final[df_final['lingua'] == i])

total_linguas = total_outras + total_ingles

In [ ]:
print ('total_outras=',total_outras, 'total_ingles=', total_ingles, '\n Total linguas', total_linguas, '\n diferença ingles - outras= ', total_ingles - total_outras, '\n registros dataset=', len(df_final.lingua))

In [ ]:
print ("Ingles=", (total_ingles/total_linguas)*100)
print ("Outras=", (total_outras/total_linguas)*100)

Percebe-se acima que mesmo somando todas as outras linguas originais dos filmes, o inglês é o maior idioma, com 70%. 

## Hipótese 3 - As maiores média de votos está relacionada a algum cia de produção?

In [ ]:
df_final.columns

In [ ]:
print ('MGM', len(df[df['Metro-Goldwyn-Mayer (MGM)'] == True]))
print ('Warner', len(df[df['Warner Bros.'] == True]))
print ('Paramount', len(df[df['Paramount Pictures'] == True]))
print ('Fox', len(df[df['Twentieth Century Fox Film Corporation'] == True]))
print ('Universal', len(df[df['Universal Pictures'] == True]))
print ('RKO', len(df[df['RKO Radio Pictures'] == True]))
print ('Columbia', len(df[df['Columbia Pictures'] == True]))
print ('Mosfilm', len(df[df['Mosfilm'] == True]))
print ('Walt Disney', len(df[df['Walt Disney Pictures'] == True]))
print ('Universal International Pictures (UI)', len(df[df['Universal International Pictures (UI)'] == True]))

In [ ]:
# plotando os maiores dos gêneros obtidos nos gráficos acima: comédia, suspense, romance e drama

# Como são várias as features de gênero, as divide
x = df_final.groupby(['media_votos']).sum()

x['Metro-Goldwyn-Mayer (MGM)'].plot(kind = 'line',color='pink',grid=True, stacked=True,label='MGM')
x['Warner Bros.'].plot(kind = 'line',color='blue',grid=True, stacked=True,label='Warner Bros.')
x['Paramount Pictures'].plot(kind = 'line',color='magenta',grid=True, stacked=True,label='Paramount Pictures')
x['Twentieth Century Fox Film Corporation'].plot(kind = 'line',color='red', grid=True, stacked=True,label='Twentieth Century Fox Film Corporation')
x['Universal Pictures'].plot(kind = 'line',color='yellow', grid=True, stacked=True,label='Universal Pictures')
x['RKO Radio Pictures'].plot(kind = 'line',color='brown', grid=True, stacked=True,label='RKO Radio Pictures')
x['Columbia Pictures'].plot(kind = 'line',color='black', grid=True, stacked=True,label='Columbia Pictures')
x['Mosfilm'].plot(kind = 'line',color='green', grid=True, stacked=True,label='Mosfilm')
x['Walt Disney Pictures'].plot(kind = 'line',color='purple', grid=True, stacked=True,label='Walt Disney Pictures')
x['Universal International Pictures (UI)'].plot(kind = 'line',color='grey', grid=True, stacked=True,label='Universal International Pictures (UI)')

plt.legend()
plt.title('Companias x Média Votos')
plt.xlabel("Companias")
plt.ylabel("Média Votos")

In [ ]:
df.columns

In [ ]:
# plotando os maiores dos gêneros obtidos nos gráficos acima: comédia, suspense, romance e drama

# Como são várias as features de gênero, as divide
x = df_final.groupby(['media_votos']).sum()


x['Metro-Goldwyn-Mayer (MGM)'].plot(kind = 'bar',color='pink',grid=True, stacked=True,label='MGM')
x['Warner Bros.'].plot(kind = 'bar',color='blue',grid=True, stacked=True,label='Warner Bros.')
x['Paramount Pictures'].plot(kind = 'bar',color='magenta',grid=True, stacked=True,label='Paramount Pictures')
x['Twentieth Century Fox Film Corporation'].plot(kind = 'bar',color='red', grid=True, stacked=True,label='Twentieth Century Fox Film Corporation')
x['Universal Pictures'].plot(kind = 'bar',color='yellow', grid=True, stacked=True,label='Universal Pictures')
x['RKO Radio Pictures'].plot(kind = 'bar',color='brown', grid=True, stacked=True,label='RKO Radio Pictures')
x['Columbia Pictures'].plot(kind = 'bar',color='black', grid=True, stacked=True,label='Columbia Pictures')
x['Mosfilm'].plot(kind = 'bar',color='green', grid=True, stacked=True,label='Mosfilm')
x['Walt Disney Pictures'].plot(kind = 'bar',color='purple', grid=True, stacked=True,label='Walt Disney Pictures')
x['Universal International Pictures (UI)'].plot(kind = 'bar',color='grey', grid=True, stacked=True,label='Universal International Pictures (UI)')


plt.legend()
plt.title('Companias x Média Votos')
plt.xlabel("Companias")
plt.ylabel("Média Votos")

Percebe-se que a companhia com as maiores médias de votos é a Warner

### 5) Modelagem 30%

Agora você terá mais uma oportunidade de mostrar o que você aprendeu durante o módulo 2 quanto a modelagem de dados e criação de modelos. Utilizando os dados preparados na seção 1 e após a análise feita na seção 2 você deverá:
1. **Defina um problema de regressão ou classificação que envolva uma variável dos seus dados.**

***---> Comente brevemente sua decisão aqui.***

Será criado um modelo de regrassão para prever a média de votos dos filmes.

2. **Realize ao menos 2 técnicas de processamento e seleção de features.**
    * Isto inclui, normalização, PCA, e técnicas de seleção de features como information gain. Seja criativo pois está parte é crucial para seu modelo. Você pode escolhar manualmente as features desde que seja justificada na seção 2 (parte de Análise).


### DISCRETIZAÇÃO DA FEATURE DURACAO

In [ ]:
# verificando a correlação da duração com a média de votos
corr_ini_duracao = df['duracao'].corr(df['media_votos'])
print ('duração x media_votos',corr_ini_duracao)

In [ ]:
df_final.duracao.dtype

In [ ]:
# Vou tentar criar grupos para definir a feture e verificar se há melhora na correlação
intervals = pd.qcut(df_final['duracao'], 2).astype(str).unique()
intervals

In [ ]:
df_final['tipo_duracao'] = pd.qcut(df_final['duracao'], 2).astype(str).map( {
                            intervals[0]: 'media',
                            intervals[1]: 'longa'
                            } ).astype(str)
df_final[['duracao','tipo_duracao']]

In [ ]:
# Convertando strings em numéricos usando dicionário

df_final['tipo_duracao'] = df_final['tipo_duracao'].map({
'media': 0, 
'longa': 1})

In [ ]:
# verificando a correlação da duração com a média de votos
corr_fim_duracao = df_final['tipo_duracao'].corr(df_final['media_votos'])
print ('Tipo duração x media_votos',corr_fim_duracao)

In [ ]:
print ('Houve uma pequena melhorada na correção', corr_fim_duracao - corr_ini_duracao)

Fiz o teste da correção para os valores de 2 a 5. Quanto maior a qtde de grupos, pior a correlação.

### DISCRETIZAÇÃO DA FEATURE ANO_LANCAMENTO

In [ ]:
# verificando a correlação da duração com a média de votos
corr_ini_anoLancamento = df_final['ano_lancamento'].corr(df_final['media_votos'])
print ('Ano lançamento x media_votos',corr_ini_anoLancamento)

In [ ]:
# tentar criar períodos para melhor a correlação
# avaliando a quantidade de períodos a serem criados 
intervals = pd.qcut(df_final['ano_lancamento'], 10).astype(str).unique()
intervals

In [ ]:
# criando os perídos para definir o ano de lançamento dos filmes
df_final['periodo'] = pd.qcut(df_final['ano_lancamento'], 10).astype(str).map({intervals[0]: '1993 a 2000',
        intervals[1]: '1970 a 1982',
        intervals[2]: '1982 a 1993',
        intervals[3]: '1952 a 1970',
        intervals[4]: '1873 a 1952',
        intervals[5]: '2000 a 2005',
        intervals[6]: '2009 a 2012',
        intervals[7]: '2012 a 2014',
        intervals[8]: '2005 a 2009',
        intervals[9]: '2014 a 2020'} ).astype(str)

df_final[['ano_lancamento','periodo']]

In [ ]:
# Convertando strings em numéricos usando dicionário

df_final['tipo_duracao'] = df_final['periodo'].map({
    '1993 a 2000': 0,
    '1970 a 1982': 1,
    '1982 a 1993': 2,
    '1952 a 1970': 3,
    '1873 a 1952': 4,
    '2000 a 2005': 5,
    '2009 a 2012': 6,
    '2012 a 2014': 7,
    '2005 a 2009': 8,
    '2014 a 2020': 9
})

In [ ]:
# verificando a correlação da duração com a média de votos
corr_fim_anoLancamento = df_final['tipo_duracao'].corr(df_final['media_votos'])
print ('Ano lançamento x media_votos',corr_fim_anoLancamento)

In [ ]:
print (" final= ", corr_fim_anoLancamento, "| inicial = ",corr_ini_anoLancamento)

In [ ]:
# Houve melhora então a nova feature será mantida

### FAZER A CORRELAÇÃO DAS FEATURES A SEREM TRABALHADAS

In [ ]:
# verificando a correlação dos generos

print ('acao x media_votos',df_final['acao'].corr(df_final['media_votos']))
print ('aventura x media_votos',df_final['aventura'].corr(df_final['media_votos']))
print ('animacao x media_votos',df_final['animacao'].corr(df_final['media_votos']))
print ('comedia x media_votos',df_final['comedia'].corr(df_final['media_votos']))
print ('crime x media_votos',df_final['crime'].corr(df_final['media_votos']))
print ('documentario x media_votos',df_final['documentario'].corr(df_final['media_votos']))
print ('drama x media_votos',df_final['drama'].corr(df_final['media_votos']))
print ('familia x media_votos',df_final['familia'].corr(df_final['media_votos']))
print ('fantasia x media_votos',df_final['fantasia'].corr(df_final['media_votos']))
print ('estrangeiro x media_votos',df_final['estrangeiro'].corr(df_final['media_votos']))
print ('historia x media_votos',df_final['historia'].corr(df_final['media_votos']))
print ('horror x media_votos',df_final['horror'].corr(df_final['media_votos']))
print ('musical x media_votos',df_final['musical'].corr(df_final['media_votos']))
print ('misterio x media_votos',df_final['misterio'].corr(df_final['media_votos']))
print ('romance x media_votos',df_final['romance'].corr(df_final['media_votos']))
print ('ficcao_cientifica x media_votos',df_final['ficcao_cientifica'].corr(df_final['media_votos']))
print ('filmes_tv x media_votos',df_final['filmes_tv'].corr(df_final['media_votos']))
print ('suspense x media_votos',df_final['suspense'].corr(df_final['media_votos']))
print ('gerra x media_votos',df_final['gerra'].corr(df_final['media_votos']))
print ('ocidental x media_votos',df_final['ocidental'].corr(df_final['media_votos']))

In [ ]:
#Excluir as features com correlação negativa
'''acao x media_votos -0.08289026991958133
aventura x media_votos -0.02794159637963377
comedia x media_votos -0.01778207543223205
familia x media_votos -0.01386665979818757
fantasia x media_votos -0.013615448225225833
estrangeiro x media_votos -0.0045234706117887975
horror x media_votos -0.18819159408147684
misterio x media_votos -0.008024703581022141
ficcao_cientifica x media_votos -0.11295899314774636
filmes_tv x media_votos -0.03709269978090582
suspense x media_votos -0.09543228084790208
ocidental x media_votos -0.03503105661478576'''

df_final.drop(['acao','aventura','comedia', 'familia', 'fantasia', 'estrangeiro', 'horror', 'misterio',
    'ficcao_cientifica', 'filmes_tv', 'suspense', 'ocidental'], axis = 1, inplace = True)

In [ ]:
print ('video x media_votos',df_final['video'].corr(df_final['media_votos']))

In [ ]:
# remoção da feature video
df_final.drop(['video'], axis = 1, inplace = True)

In [ ]:
df_final.columns

In [ ]:
# features de companhias de produção
''''Metro-Goldwyn-Mayer (MGM)','Warner Bros.', 'Paramount Pictures',
       'Twentieth Century Fox Film Corporation', 'Universal Pictures',
       'RKO Radio Pictures', 'Columbia Pictures', 'Mosfilm',
       'Walt Disney Pictures', 'Universal International Pictures (UI)'
       '''

print ('MGV',df_final['Metro-Goldwyn-Mayer (MGM)'].corr(df_final['media_votos']))
print ('Warner Bros. x media_votos',df_final['Warner Bros.'].corr(df_final['media_votos']))
print ('Paramount x media_votos',df_final['Paramount Pictures'].corr(df_final['media_votos']))
print ('Fox x media_votos',df_final['Twentieth Century Fox Film Corporation'].corr(df_final['media_votos']))
print ('Universal Pictures x media_votos',df_final['Universal Pictures'].corr(df_final['media_votos']))
print ('RKO Radio Pictures x media_votos',df_final['RKO Radio Pictures'].corr(df_final['media_votos']))
print ('Columbia Pictures x media_votos',df_final['Columbia Pictures'].corr(df_final['media_votos']))
print ('Mosfilm x media_votos',df_final['Mosfilm'].corr(df_final['media_votos']))
print ('Walt Disney Pictures x media_votos',df_final['Walt Disney Pictures'].corr(df_final['media_votos']))
print ('Universal International Pictures (UI) x media_votos',df_final['Universal International Pictures (UI)'].corr(df_final['media_votos']))

In [ ]:
# excluido das features com correlação negativa
'''
MGV -0.005643144906661703
RKO Radio Pictures x media_votos -0.005677699693419887

'''

df_final.drop(['Metro-Goldwyn-Mayer (MGM)','RKO Radio Pictures'], axis = 1, inplace = True)

In [ ]:
# features de paizes onde os filmes foram produzidos
'''
'United States of America', 'United Kingdom', 'France', 'Japan',
       'Italy', 'Canada', 'Canada.1', 'Canada.2', 'Canada.3', 'Germany',
       'Russia', 'India', 'Canada.4', 'Canada.5', 'Canada.6', 'Canada.7',
       'English', 'Français', '日本語', 'Italiano', 'Español'
    '''
print ('United States of America',df_final['United States of America (pais producao)'].corr(df_final['media_votos']))
print ('United Kingdom x media_votos',df_final['United Kingdom (pais producao)'].corr(df_final['media_votos']))
print ('France x media_votos',df_final['France (pais producao)'].corr(df_final['media_votos']))
print ('Japan x media_votos',df_final['Japan (pais producao)'].corr(df_final['media_votos']))
print ('Italy x media_votos',df_final['Italy (pais producao)'].corr(df_final['media_votos']))
print ('Canada x media_votos',df_final['Canada (pais producao)'].corr(df_final['media_votos']))


In [ ]:
# remoção das features com correlação negativa
'''
United States of America -0.10648717484568719
Canada x media_votos -0.04293418732490301
'''

df_final.drop(['United States of America (pais producao)','Canada (pais producao)'], axis = 1, inplace = True)

In [ ]:
df_final.columns

In [ ]:
# correlação dos idimos de tradução
''' 'English', 'Français', '日本語', 'Italiano', 'Español' '''

print ('English x media_votos',df_final['English'].corr(df_final['media_votos']))
print ('Français x media_votos',df_final['Français'].corr(df_final['media_votos']))
print ('日本語 x media_votos',df_final['日本語'].corr(df_final['media_votos']))
print ('Italiano x media_votos',df_final['Italiano'].corr(df_final['media_votos']))
print ('Español x media_votos',df_final['Español'].corr(df_final['media_votos']))

In [ ]:
# remoção das features com correlação negativa
'''
English x media_votos -0.1104885381792159
'''

df_final.drop(['English'], axis = 1, inplace = True)

### FAZER A NORMALIZAÇÃO DOS DADOS

### APLICAR O PCA

3. **Defina uma métrica para avaliar o seu modelo.**
    * Por exemplo, você pode utilizar MAE (Mean Absolute Error) para um problema de regressão. Ou, F1-Score para um problema de classificação. Há varias métricas, então escolha sabiamente.


***---> Comente brevemente sua decisão aqui.***

4. **Divida seus dados em 2 conjuntos. Um de treino e outro conjunto de teste.**
    * Treine e otimize seu modelo no conjunto de treino e apenas use o conjunto de teste para apresentar os resultados finais.


5. **Treine um ou mais modelos de ML para o seu problema.**
    * Escolha 1 ou mais tipos de classificadores ou regressores dependendo do seu problema.
    * Por exemplo, TreeClassifier para um problema de classificação. 
    * Use cross-validation e outras técnicas como GridSearch e ou RandomizedSearch para encontrar os melhores parametros para o seu modelo.


6. **Apresente (com visualizações) os resultados do seu modelo mostrando que ele é melhor do que um baseline não aleatório.**
    * Para o baseline, você pode escolher um modelo bem trivial mas não aleatório. Por exemplo, para um problema de classificação um baseline pode ser a classe mais presente caso o conjunto de dados seja desbalanceado.  Um modelo mais simples também pode ser utilizado como baseline, por exemplo, você escolheu um Random Forest Classifier, e comparou os resultados um Logistic Regression. Você pode até mesmo escolher um modelo de AutoML(como TPOT) como Baseline.

### 6) Conclusões **10%**

*Partindo das suas hipoteses e investigações, o que você consegue concluir? Suas hipoteses se concretizaram?*